# Werewolf environment demo

This notebook shows how to run a lightweight Werewolf game step, inspect deception labels (`deceptive` + `naturally_deceptive`), and feed outputs into the existing sentence localization pipeline.


In [ ]:
from pathlib import Path
import sys
sys.path.append(str(Path('..') / 'src'))

from werewolf_environment import WerewolfEnvironment


In [ ]:
class ScriptedAgent:
    def __init__(self, name, scripted_actions):
        self.name = name
        self.scripted_actions = scripted_actions
        self.idx = 0
        self.reasoning_instruction = 'COD'

    def chat(self, messages, num_responses=1):
        out = self.scripted_actions[min(self.idx, len(self.scripted_actions)-1)]
        self.idx += 1
        return [out]


In [ ]:
agents = [
    ScriptedAgent('Alice', [{"Action": "SPEAK", "Target": "Bob", "Claim": "villager", "Statement": "Bob is safe."}, {"Action": "VOTE", "Target": "Carol"}]),
    ScriptedAgent('Bob',   [{"Action": "SPEAK", "Target": "Alice", "Claim": "werewolf", "Statement": "Alice is suspicious."}, {"Action": "VOTE", "Target": "Alice"}]),
    ScriptedAgent('Carol', [{"Action": "SPEAK", "Target": "Bob", "Claim": "werewolf", "Statement": "I think Bob is wolf."}, {"Action": "VOTE", "Target": "Bob"}]),
]
env = WerewolfEnvironment(agents, seed=7)
env.roles  # hidden ground-truth (for mining/eval only)


In [ ]:
state = env.get_state()
play = env.step()
play


In [ ]:
# Labels emitted on discuss actions
print('deceptive:', play['naturally_deceptive'])
print('truth_context:', env.truth_context())


## Run Werewolf mining (same workflow style as BS)

```bash
python BS/src/werewolf_deception_miner.py \
  --model_name deepseek-ai/DeepSeek-R1-Distill-Qwen-7B \
  --is_reasoning_model \
  --output_dir BS/Results/WerewolfDeceptionMining/demo_run \
  --num_players 5 --max_games 5 --max_turns 20
```


## Build sentence dataset + localize deception

The output `deception_samples.jsonl` remains compatible with the existing sentence pipeline.

```bash
python BS/src/build_sentence_dataset.py \
  --input_root BS/Results/WerewolfDeceptionMining \
  --out_dir BS/Results/SentencePipeline/werewolf_v1 \
  --text_field action_reasoning --include_messages

python BS/src/sentence_localization_batch.py \
  --examples_path BS/Results/SentencePipeline/werewolf_v1/examples.jsonl \
  --sentences_path BS/Results/SentencePipeline/werewolf_v1/sentences.jsonl \
  --jsonl_path BS/Results/SentencePipeline/werewolf_v1/localization.jsonl \
  --model_name deepseek-ai/DeepSeek-R1-Distill-Qwen-7B
```
